Deze notebook is bedoelt om een rasterdataset op te knippen met een feature layer. Hierbij wordt ook een deel van de raster boven de waarde "n" omgezet naar "NoData". Maak van tevoren een map aan in je workspace genaamd DEM_CLIP

In [7]:
import arcpy, os
import numpy as np
from osgeo import gdal
arcpy.env.addOutputsToMap = False   # Prevents many small grid files to show on map
arcpy.env.overwriteOutput = True

# Directory option 1: Use project gdb
# Directory option 2: use project directory
# Directory option 3: Custom directory
Dir = 2  # [1, 2 or 3] (Default is 1)

if Dir == 1:
    p = arcpy.mp.ArcGISProject("CURRENT")
    directory = p.defaultGeodatabase   
elif Dir == 2:
    directory = os.path.dirname( arcpy.mp.ArcGISProject("CURRENT").filePath )
elif Dir == 3:
    directory = r"H:\DATA\Service\Wietse\GIS\DEM"

    
#De Shapefile die je wilt gebruiken voor het opknippen van de raster (BR_Peilgebieden, BR_Afvoergebieden, ect.)
clipShapefile  = "BR_Peilgebieden_Filter_West_HDSR"

#De raster die je wilt gebruiken als input (AHN3, AHN4, ect.)
#Hou rekening met het Talut van de watergangen en neem indien nodig stappen om deze uit de DEM te filteren
rasterlist = ["AHN3_HDSR_FILL_ISV_NO.TIF"]


In [8]:
Percentile_value_min = 10            # Het Percentiel [0-100] waaronder je alle data wilt omzetten naar nodata [Default = 0]
Percentile_value     = 20              # Het Percentiel [0-100] waarboven je alle data wilt omzetten naar nodata [Default = 10]
Raster_Clip          = "BR_West_04042023_5"   # De folder waar de geclipte raster worden opgeslagen
Raster_Con_Clip      = "STAT_10_20"       # De folder waar de geclipte rasters met NoData filter worden opgeslagen

arcpy.management.CreateFolder(directory, Raster_Clip)
outputWorkspace = directory 
arcpy.env.workspace = outputWorkspace
outputWorkspace_clip = os.path.join(outputWorkspace, Raster_Clip)

In [9]:
clipShapes = set()
with arcpy.da.SearchCursor(clipShapefile, ['OID@']) as cursor:
    for row in cursor:
        clipShapes.add(row[0])

for raster in rasterlist:
    rastername, rasterext = os.path.splitext(raster)
    for i in clipShapes:
        newRaster = "{}_clip_{}.tif".format(rastername, i)
        newRasterPath = os.path.join(outputWorkspace_clip, newRaster)
        if arcpy.Exists('clipLayer'): # Remove existing 'clipLayer' feature layer if it still exists for some reason
            arcpy.Delete_management('clipLayer')
        arcpy.MakeFeatureLayer_management(clipShapefile, 'clipLayer', ' "FID" = {}'.format(i)) #create a layer with only polygon i
        arcpy.Clip_management(raster, "#", newRasterPath, 'clipLayer', "99", "ClippingGeometry") #clip based on layer, clipping geometry will use the polygon extent only
        arcpy.Delete_management('clipLayer')


In [10]:
#Deze Functie berekend het percentiel
def Percentile(rasterlayername,n,folder,dt = "f4"):  
    newRasterPath = os.path.join(folder, rasterlayername)
    ds = gdal.Open(newRasterPath)
    band = ds.GetRasterBand(1)
    arr = np.array(band.ReadAsArray(), dtype= dt)
    arr[arr==99] = np.nan
    Perc = np.nanpercentile(arr,n)
    
    return Perc


arcpy.env.workspace = outputWorkspace_clip
arcpy.management.CreateFolder(outputWorkspace_clip, Raster_Con_Clip)
outputWorkspace_con = os.path.join(outputWorkspace_clip,Raster_Con_Clip) 

rasterlist = arcpy.ListRasters('*.tif*') # Get a list of input rasters

for raster in rasterlist:
    try:
        Raster_tmp = arcpy.sa.Raster(raster)
        Perc = Percentile(raster,Percentile_value,outputWorkspace_clip)
        Perc_min = Percentile(raster,Percentile_value_min,outputWorkspace_clip)
        raster_con = arcpy.sa.SetNull(Raster_tmp, Raster_tmp, "VALUE < {} OR VALUE > {}".format(Perc_min, Perc))
        #raster_con = arcpy.sa.SetNull(Raster_tmp > Perc, Raster_tmp)
        #raster_con = arcpy.sa.SetNull(Raster_tmp < Perc_min, Raster_tmp)
        raster_con.save(outputWorkspace_con + '\_con' + raster)
        del raster_con
    except MemoryError:
        print(raster + " is too big")
        continue
    except:
        print(raster + " something else went wrong")
        continue
        
    

###### 